<a href="https://colab.research.google.com/github/Naitsirc98/Dog-breed-recognition/blob/master/Dog_breed_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de razas de perros

Vamos a desarrollar una red que sea capaz de clasificar entre razas de perros distintas. El dataset es el [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/main.html), reducido a 12 clases.



Como tengo el dataset descargado en mi drive, ejecutamos lo siguiente para acceder:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Comprobamos que GPU estamos utilizando

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Cargando los datos

Vamos a desarrollar una función que nos cargue los datos necesarios y nos genere los generadores de entrenamiento y test:

In [0]:
from keras.preprocessing.image import ImageDataGenerator

def load_data(batch_size = 20, directory = '/content/drive/My Drive/Colab Notebooks/datasets/Dogs/Images'):

    train_datagen = ImageDataGenerator(
        rescale = 1.0 / 255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True
    )

    test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

    train_generator = train_datagen.flow_from_directory(
        directory,
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        directory,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

## Modelo

Creamos el modelo con Keras:

In [0]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Activation, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

train_gen, test_gen = load_data()

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))

model.summary()


## O cargamos el modelo desde el disco

In [0]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Activation, Dense
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

from keras.models import model_from_json

train_gen, test_gen = load_data()

# Model reconstruction from JSON file
with open("/content/drive/My Drive/Colab Notebooks/modelb.json", 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights("/content/drive/My Drive/Colab Notebooks/modelb.h5")

model.summary()

## Entrenamiento

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

for i in range(0, 10):
  epochs = 5

  history = model.fit_generator(
        train_gen,
        steps_per_epoch=100,
        epochs=epochs,
        validation_data=test_gen,
        validation_steps=800,
  )

  # serialize model to JSON
  model_json = model.to_json()
  with open("/content/drive/My Drive/Colab Notebooks/modelb.json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights("/content/drive/My Drive/Colab Notebooks/modelb.h5")
  print("Saved model to disk")

Epoch 1/5
100/100 [==============================] - 166s 2s/step - loss: 0.2026 - acc: 0.9325 - val_loss: 0.1394 - val_acc: 0.9547
Epoch 2/5
100/100 [==============================] - 160s 2s/step - loss: 0.1600 - acc: 0.9499 - val_loss: 0.1295 - val_acc: 0.9617
Epoch 3/5
100/100 [==============================] - 159s 2s/step - loss: 0.1769 - acc: 0.9435 - val_loss: 0.1216 - val_acc: 0.9615
Epoch 4/5
100/100 [==============================] - 159s 2s/step - loss: 0.1865 - acc: 0.9410 - val_loss: 0.3118 - val_acc: 0.9075
Epoch 5/5
100/100 [==============================] - 157s 2s/step - loss: 0.1145 - acc: 0.9604 - val_loss: 0.1462 - val_acc: 0.9447
Saved model to disk
Epoch 1/5
100/100 [==============================] - 163s 2s/step - loss: 0.1198 - acc: 0.9600 - val_loss: 0.1846 - val_acc: 0.9397
Epoch 2/5
100/100 [==============================] - 157s 2s/step - loss: 0.1086 - acc: 0.9655 - val_loss: 0.0481 - val_acc: 0.9832
Epoch 3/5
 99/100 [============================>.] - ETA